# runtime.net_process

> TODO fill in description

In [ ]:
#| default_exp runtime.node_process

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
import asyncio

import fbdev
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection, PortID, PortCollection
from fbdev.graph.net import BaseNode
from fbdev.runtime import BaseRuntime

In [ ]:
#|hide
from fbdev.complib import ExecComponent
from fbdev.complib.func_component_factory import func_component

In [ ]:
#|hide
show_doc(fbdev.runtime.node_process.NodeProcess)

---

### NodeProcess

>      NodeProcess (node:fbdev.graph.net.BaseNode,
>                   stop_port:Tuple[fbdev.comp.port.PortType,str]=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class NodeProcess(BaseRuntime):
    def __init__(self, node:BaseNode, stop_port:PortID=None):
        super().__init__()
        self._node:BaseNode = node
        self._stop_port = stop_port
        self._stop_listener_task = None
        
        if self._stop_port is not None and self._stop_port not in self._node.ports:
            raise ValueError(f"Port {self._stop_port} does not exist in node.")

    @property
    def ports(self) -> PortCollection: return self._node.ports

    def start(self):
        """Note: this method cannot be run from within an event loop."""
        super().start()
        raise NotImplementedError(f"{self.__class__.__name__} does not support synchronous execution.")
    
    async def astart(self):
        await super().astart()
        
        if self._stop_port is not None:
            async def stop_listener():
                try:
                    await self._node.ports[self._stop_port].get()
                    if not self._stopped: await self.stop()
                except asyncio.CancelledError: pass
            self._stop_listener_task = asyncio.create_task(stop_listener())
            
        await self._node.task_manager.exec_coros(self._node.start())
        self._started = True
    
    async def await_stop(self):
        # Tried doing this by creating an event self._stop_event.
        # For some reason this would cause all tasks to just hang forever.
        # So instead, we just wait for the node to stop.
        # Really perplexing...
        await self._node.task_manager.exec_coros(self._node.states.stopped.wait(True), print_all_exceptions=False)
    
    async def stop(self):
        await super().stop()
        if self._stop_listener_task is not None:
            self._stop_listener_task.cancel()
            try: await self._stop_listener_task
            except asyncio.CancelledError: pass
        await self._node.task_manager.exec_coros(self._node.stop(), print_all_exceptions=False)
        self._stopped = True
        
    async def await_message(self, name:str):
        await self._node.task_manager.exec_coros(self._node.await_message(name), print_all_exceptions=False)

In [ ]:
class FooComponent(ExecComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.MESSAGE, 'stop'),
    )
    async def _execute(self):
        print("Stopping")
        await self.send_message('stop')
        
async with NodeProcess.from_component(FooComponent, stop_port=(PortType.MESSAGE, 'stop')) as ex:
    await ex.astart()
    await ex.await_stop()

RuntimeError: NodeProcess has not yet been started.

In [ ]:
@func_component(loop_execution=True)
def SquareComponent(inp):
    return inp**2

async with NodeProcess.from_component(SquareComponent) as ex:
    await ex.astart()
    
    await ex.ports.input.inp.put_value(2)
    print("2^2 =",await ex.ports.output.out.get_and_consume())
    
    await ex.ports.input.inp.put_value(5)
    print("5^2 =",await ex.ports.output.out.get_and_consume())
    
    await ex.ports.input.inp.put_value(7)
    print("7^2 =",await ex.ports.output.out.get_and_consume())

2^2 = 4
5^2 = 25
7^2 = 49


In [ ]:
@func_component(loop_execution=False)
def ExceptionComponent() -> None:
    raise Exception()

try:
    async with NodeProcess.from_component(ExceptionComponent) as ex:
        await ex.aexecute()
        await ex.await_message('executed')
except Exception as e:
    print("Exception raised")

Exception raised
